In [ ]:
!pip install mplfinance

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
 Ativo = "PETR4.SA"
 dados = yf.download(Ativo, start="2024-01-01", end="2024-03-27")
 dados

In [ ]:
dados.columns

In [8]:
dados.columns = ["Abertura", "Max", "Min", "Fechamento", "Fech_adjust", "Volume"]
dados = dados.rename_axis("Data")


In [ ]:
dados["Fechamento"].plot(figsize=(10,6))
plt.title("Variação do preço x data", fontsize=16)
plt.legend("Fechamento")

In [ ]:
df = dados.head(60)
df["Data"] = df.index
df["Data"] = df["Data"].apply(mdates.date2num)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
width = 0.7
for i in range(len(df)):
  if df["Fechamento"].iloc[i] > df["Abertura"].iloc[i]:
    color = "green"
  else:
    color = "red"

  ax.plot([df["Data"].iloc[i], df["Data"].iloc[i]],
        [df["Min"].iloc[i], df["Max"].iloc[i]],
        color = color,
        linewidth=1)

  ax.add_patch(plt.Rectangle((df["Data"].iloc[i] - width/2, min(df["Abertura"].iloc[i], df["Fechamento"].iloc[i])),
                              width,
                              abs(df["Fechamento"].iloc[i] - df["Abertura"].iloc[i]),
                                  facecolor = color))

df["MA7"] = df["Fechamento"].rolling(window=7).mean()
df["MA14"] = df["Fechamento"].rolling(window=14).mean()
ax.plot(df["Data"], df["MA7"], color="orange", label="Média móvel 7 dias")
ax.plot(df["Data"], df["MA14"], color="purple", label="Média móvel 14 dias")
ax.legend()
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
plt.xticks(rotation=45)

plt.title(f"Gráfico de Candlestick - {Ativo} com matplotlib")
plt.xlabel("Data")
plt.ylabel("Preço")

plt.grid(True)

plt.show()

In [15]:
fig = make_subplots(rows=2,cols=1, shared_xaxes=True,
										vertical_spacing=0.1,
										subplot_titles=("Candlestick", "Volume transacionado"),
										row_width=[0.2, 0.7])


fig.add_trace(go.Candlestick(x=df.index,
														 open=df["Abertura"],
														 high=df["Max"],
														 low=df["Min"],
														 close=df["Fechamento"],
														 name="Candlestick"),
														 row=1, col=1)

#ADIÇÃO DAS MÉDIAS MÓVEIS
fig.add_trace(go.Scatter(x=df.index,
												 y=df["MA7"],
												 mode="lines",
												 name="MA7 - Média móvel 7 dias"),
												 row=1, col=1)

fig.add_trace(go.Scatter(x=df.index,
												 y=df["MA14"],
												 mode="lines",
												 name="MA14 - Média móvel 14 dias"),
												 row=1, col=1)

#ADIÇÃO VOLUME DE TRANSAÇÃO
fig.add_trace(go.Bar(x=df.index,
										 y=df["Volume"],
										 name="Volume"),
										 row=2, col=1)
fig.update_layout(yaxis_title="Preço",
									xaxis_rangeslider_visible=False,
									width=1100, height=600)
fig.show()